In [1]:
import requests
from requests_oauthlib import OAuth1

In [2]:
import urllib
import cnfg
from pprint import pprint
config = cnfg.load(".twitter_config")

In [3]:
oauth = OAuth1(config["consumer_key"],
               config["consumer_secret"],
               config["access_token"],
               config["access_token_secret"])

In [4]:
import json
from pymongo import MongoClient

client = MongoClient('ec2-34-198-179-91.compute-1.amazonaws.com', 27017)
db = client.fletcher
chegg_col = db.chegg

In [13]:
main_keys = ['created_at', 'id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweet_count', 'text']
user_keys = ['created_at', 'description', 'followers_count', 'friends_count', 'id', 'location', 'name', 'screen_name', 'time_zone']
def insertTweet(collection, tweet):
    data = {}
    for key in main_keys:
        data[key] = tweet[key]
    for key in user_keys:
        data["user_" + key] = tweet['user'][key]
    collection.insert_one(data)
        

In [23]:
parameters = {"q": "to%3Achegg", "count":100, "lang":"en", "max_id": "830484841569320960", "result_type": "recent"}
response = requests.get("https://api.twitter.com/1.1/search/tweets.json",
                        params = parameters,
                        auth=oauth)

In [24]:
n_params = response.json()["search_metadata"]['next_results']
print(n_params)

KeyError: 'next_results'

In [14]:
twitter_response = response.json()['statuses']
for tweet in twitter_response:
    insertTweet(chegg_col, tweet)

In [15]:
import time

In [16]:
for i in range(5):
    for i in range(49):
        response = requests.get("https://api.twitter.com/1.1/search/tweets.json" + n_params, auth=oauth)
        n_params = response.json()["search_metadata"]["next_results"]
        twitter_response = response.json()['statuses']
        for tweet in twitter_response:
            insertTweet(chegg_col, tweet)
        print(n_params)
    time.sleep(15*60) # sleep for 15 minutes

?max_id=832808023185772545&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=832443541066543104&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=832242043929972737&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=831736450232287235&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=831363777773740032&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=830995503844298751&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=830792520724541440&q=to%253Achegg&lang=en&count=100&include_entities=1
?max_id=830484841569320960&q=to%253Achegg&lang=en&count=100&include_entities=1


KeyError: 'next_results'

In [20]:
chegg_col.count()

900

In [41]:
cur = db.chegg.find({"user_screen_name" : "blakeshelton"})